<a href="https://colab.research.google.com/github/5r33kanth/Lip_Reading-main-/blob/main/Lip%20reading%20new%20final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI POWERED LIP READING SYSTEM v_2


In [27]:
%cd "/content/"
!git clone https://github.com/mpc001/Visual_Speech_Recognition_for_Multiple_Languages.git
%cd "Visual_Speech_Recognition_for_Multiple_Languages"

/content
fatal: destination path 'Visual_Speech_Recognition_for_Multiple_Languages' already exists and is not an empty directory.
/content/Visual_Speech_Recognition_for_Multiple_Languages


In [28]:
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install scipy
!pip install scikit-image
!pip install av
!pip install six
!pip install mediapipe
!pip install ffmpeg-python

## Video preparation

1. Download a video.

In [29]:
!mkdir -p /content/data/
# Use wget to download the file
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1TIg8lYzeJfFrCZmksmDS_JUDx4wHrNjH' -O '/content/data/noisy_clip.mp4'
#drive.google.com/uc?export=download&id=1EpEhV_XP9gLe-ZTnoFRw-X4RGcmwH3xa

--2024-05-16 13:52:46--  https://drive.google.com/uc?export=download&id=1TIg8lYzeJfFrCZmksmDS_JUDx4wHrNjH
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.100, 142.251.2.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1TIg8lYzeJfFrCZmksmDS_JUDx4wHrNjH&export=download [following]
--2024-05-16 13:52:46--  https://drive.usercontent.google.com/download?id=1TIg8lYzeJfFrCZmksmDS_JUDx4wHrNjH&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 859421 (839K) [video/mp4]
Saving to: ‘/content/data/noisy_clip.mp4’

/content/data/noisy 100%[===================>] 839.28K  --.-KB/s    in 0

In [30]:
from IPython.display import HTML
from base64 import b64encode

## play_video function based on: https://colab.research.google.com/drive/1bNXkfpHiVHzXQH8WjGhzQ-fsDxolpUjD

def play_video(video_path, width=200):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML(f"""
  <video width={width} controls>
        <source src="{data_url}" type="video/mp4">
  </video>
  """)

In [31]:
play_video('/content/data/noisy_clip.mp4', width=300)


##  Inference pipeline


In [32]:
import os
import torch
from pipelines.model import AVSR
from pipelines.data.data_module import AVSRDataLoader
from pipelines.detectors.mediapipe.detector import LandmarksDetector

class InferencePipeline(torch.nn.Module):
    def __init__(self, modality, model_path, model_conf, detector="mediapipe", face_track=False, device="cuda:0"):
        super(InferencePipeline, self).__init__()
        self.device = device
        # modality configuration
        self.modality = modality
        self.dataloader = AVSRDataLoader(modality, detector=detector)
        self.model = AVSR(modality, model_path, model_conf, rnnlm=None, rnnlm_conf=None, penalty=0.0, ctc_weight=0.1, lm_weight=0.0, beam_size=40, device=device)
        if face_track and self.modality in ["video", "audiovisual"]:
            self.landmarks_detector = LandmarksDetector()
        else:
            self.landmarks_detector = None


    def process_landmarks(self, data_filename, landmarks_filename):
        if self.modality == "audio":
            return None
        if self.modality in ["video", "audiovisual"]:
            landmarks = self.landmarks_detector(data_filename)
            return landmarks


    def forward(self, data_filename, landmarks_filename=None):
        assert os.path.isfile(data_filename), f"data_filename: {data_filename} does not exist."
        landmarks = self.process_landmarks(data_filename, landmarks_filename)
        data = self.dataloader.load_data(data_filename, landmarks)
        transcript = self.model.infer(data)
        return transcript

    def extract_features(self, data_filename, landmarks_filename=None, extract_resnet_feats=False):
        assert os.path.isfile(data_filename), f"data_filename: {data_filename} does not exist."
        landmarks = self.process_landmarks(data_filename, landmarks_filename)
        data = self.dataloader.load_data(data_filename, landmarks)
        with torch.no_grad():
            if isinstance(data, tuple):
                enc_feats = self.model.model.encode(data[0].to(self.device), data[1].to(self.device), extract_resnet_feats)
            else:
                enc_feats = self.model.model.encode(data.to(self.device), extract_resnet_feats)
        return enc_feats

### Infer the noisy clip using an audio stream

1. Download an ASR checkpoint

In [33]:
%mkdir -p /content/data/
!wget http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_A_WER1.0.zip -O /content/data/LRS3_A_WER1.0.zip
!unzip -o /content/data/LRS3_A_WER1.0.zip -d /content/data/

--2024-05-16 13:52:48--  http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_A_WER1.0.zip
Resolving www.doc.ic.ac.uk (www.doc.ic.ac.uk)... 146.169.13.6
Connecting to www.doc.ic.ac.uk (www.doc.ic.ac.uk)|146.169.13.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 902649180 (861M) [application/zip]
Saving to: ‘/content/data/LRS3_A_WER1.0.zip’

/content/data/LRS3_ 100%[===================>] 860.83M  20.5MB/s    in 43s     

2024-05-16 13:53:31 (20.3 MB/s) - ‘/content/data/LRS3_A_WER1.0.zip’ saved [902649180/902649180]

Archive:  /content/data/LRS3_A_WER1.0.zip
  inflating: /content/data/LRS3_A_WER1.0/model.json  
  inflating: /content/data/LRS3_A_WER1.0/model.pth  


2. Build an ASR pipeline

In [34]:
modality = "audio"
model_conf = "/content/data/LRS3_A_WER1.0/model.json"
model_path = "/content/data/LRS3_A_WER1.0/model.pth"
pipeline = InferencePipeline(modality, model_path, model_conf)

3. Infer the noisy clip using the audio stream.

In [35]:
transcript = pipeline("/content/data/noisy_clip.mp4")
print(transcript)

THIS IS A PROJECT DEMO


### Infer the noisy clip using a video stream


1. Download a VSR checkpoint

In [36]:
%mkdir -p /content/data/
!wget http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_V_WER19.1.zip -O /content/data/LRS3_V_WER19.1.zip
!unzip -o /content/data/LRS3_V_WER19.1.zip -d /content/data/

--2024-05-16 13:53:46--  http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_V_WER19.1.zip
Resolving www.doc.ic.ac.uk (www.doc.ic.ac.uk)... 146.169.13.6
Connecting to www.doc.ic.ac.uk (www.doc.ic.ac.uk)|146.169.13.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 937274463 (894M) [application/zip]
Saving to: ‘/content/data/LRS3_V_WER19.1.zip’

/content/data/LRS3_ 100%[===================>] 893.85M  21.2MB/s    in 43s     

2024-05-16 13:54:31 (20.6 MB/s) - ‘/content/data/LRS3_V_WER19.1.zip’ saved [937274463/937274463]

Archive:  /content/data/LRS3_V_WER19.1.zip
  inflating: /content/data/LRS3_V_WER19.1/model.json  
  inflating: /content/data/LRS3_V_WER19.1/model.pth  


2. VSR pipeline

In [37]:
modality = "video"
model_conf = "/content/data/LRS3_V_WER19.1/model.json"
model_path = "/content/data/LRS3_V_WER19.1/model.pth"
pipeline = InferencePipeline(modality, model_path, model_conf, face_track=True)

3. Infer the noisy clip using the video stream

In [38]:
transcript = pipeline("/content/data/noisy_clip.mp4")
print(transcript)

THIS IS A PROJECT NUMBER


### Infer the noisy clip using both audio and visual streams

1. Download a AV-ASR checkpoint

In [39]:
%mkdir -p /content/data/
!wget http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_AV_WER0.9.zip -O /content/data/LRS3_AV_WER0.9.zip
!unzip -o /content/data/LRS3_AV_WER0.9.zip -d /content/data/

--2024-05-16 13:54:50--  http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_AV_WER0.9.zip
Resolving www.doc.ic.ac.uk (www.doc.ic.ac.uk)... 146.169.13.6
Connecting to www.doc.ic.ac.uk (www.doc.ic.ac.uk)|146.169.13.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1655043546 (1.5G) [application/zip]
Saving to: ‘/content/data/LRS3_AV_WER0.9.zip’

/content/data/LRS3_ 100%[===================>]   1.54G  20.8MB/s    in 76s     

2024-05-16 13:56:07 (20.7 MB/s) - ‘/content/data/LRS3_AV_WER0.9.zip’ saved [1655043546/1655043546]

Archive:  /content/data/LRS3_AV_WER0.9.zip
  inflating: /content/data/LRS3_AV_WER0.9/model.json  
  inflating: /content/data/LRS3_AV_WER0.9/model.pth  


2. Build an AV-ASR pipeline

In [40]:
modality = "audiovisual"
model_conf = "/content/data/LRS3_AV_WER0.9/model.json"
model_path = "/content/data/LRS3_AV_WER0.9/model.pth"
pipeline = InferencePipeline(modality, model_path, model_conf, face_track=True)

3. Infer the noisy clip using both audio and video streams

In [41]:
transcript = pipeline("/content/data/noisy_clip.mp4")
print(transcript)

THIS IS A PROJECT DEMO


### Crop mouth ROIs


In [42]:
import cv2
import torchvision
from pipelines.data.data_module import AVSRDataLoader
from pipelines.detectors.mediapipe.detector import LandmarksDetector

def save2vid(filename, vid, frames_per_second):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    torchvision.io.write_video(filename, vid, frames_per_second)

def preprocess_video(src_filename, dst_filename):
    landmarks = landmarks_detector(src_filename)
    data = dataloader.load_data(src_filename, landmarks)
    fps = cv2.VideoCapture(src_filename).get(cv2.CAP_PROP_FPS)
    save2vid(dst_filename, data, fps)
    return

dataloader = AVSRDataLoader(modality="video", speed_rate=1, transform=False, detector="mediapipe", convert_gray=False)
landmarks_detector = LandmarksDetector()

In [43]:
preprocess_video(src_filename="/content/data/clip.mp4", dst_filename="/content/data/roi.mp4")

In [44]:
play_video("/content/data/roi.mp4", width=300)

### Extract visual-only features

In [45]:
modality = "video"
model_conf = "/content/data/LRS3_V_WER19.1/model.json"
model_path = "/content/data/LRS3_V_WER19.1/model.pth"
pipeline = InferencePipeline(modality, model_path, model_conf, face_track=True)

[**Option 1**]. Extract features from the output of Conformer.

In [46]:
features = pipeline.extract_features("/content/data/clip.mp4")
print(features.size())

torch.Size([114, 768])


[**Option 2**]. Extract features from the output of ResNet.

In [47]:
features = pipeline.extract_features("/content/data/clip.mp4", extract_resnet_feats=True)
print(features.size())

torch.Size([114, 512])
